<a href="https://colab.research.google.com/github/Mounikab03/codtech-ML-tasks/blob/main/task_3_image_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Cell 1 — Check GPU
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

PyTorch version: 2.8.0+cu126
CUDA available: False
Device: cpu


In [2]:

!pip install -q scikit-learn

In [3]:
# Cell 3 — Fast PyTorch CNN training on CIFAR-10 (paste whole cell)
import os, time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_EPOCHS = 5       # small for quick run
LR = 1e-3
SAVE_PATH = "/content/cnn_model.pth"
NUM_WORKERS = 2

# --- Simple (fast) CNN ---
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 32x16x16
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # 64x8x8
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2) # 128x4x4
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*4*4, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.conv(x)
        x = self.classifier(x)
        return x

# --- Data ---
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])

train_set = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_set  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
classes = train_set.classes

# --- Model / Loss / Opt ---
model = SimpleCNN(num_classes=len(classes)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# --- Training loop ---
def train_one_epoch(model, loader, opt, criterion):
    model.train()
    running_loss = 0.0; correct = 0; total = 0
    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        opt.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item()*images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds==labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def eval_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    preds_all, labels_all = [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()*images.size(0)
            preds_all.append(outputs.argmax(dim=1).cpu().numpy())
            labels_all.append(labels.cpu().numpy())
    preds_all = np.concatenate(preds_all)
    labels_all = np.concatenate(labels_all)
    return running_loss/len(labels_all), preds_all, labels_all

start_time = time.time()
best_acc = 0.0
for epoch in range(1, NUM_EPOCHS+1):
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, preds, labels = eval_model(model, test_loader, criterion)
    val_acc = (preds==labels).mean()
    epoch_time = time.time()-t0
    print(f"Epoch {epoch}/{NUM_EPOCHS} — {epoch_time:.1f}s  Train loss: {train_loss:.4f} Acc: {train_acc:.4f}  |  Test loss: {val_loss:.4f} Acc: {val_acc:.4f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), SAVE_PATH)
print(f"\nDone in {(time.time()-start_time)/60:.2f} minutes. Best test acc: {best_acc:.4f}")
# --- Final metrics ---
from sklearn.metrics import classification_report, confusion_matrix
_, preds, labels = eval_model(model, test_loader, criterion)
print("\nClassification report:\n", classification_report(labels, preds, target_names=classes, zero_division=0, digits=4))
print("Confusion matrix shape:", confusion_matrix(labels, preds).shape)
print(f"\nSaved best model to: {SAVE_PATH}")

100%|██████████| 170M/170M [00:11<00:00, 15.5MB/s]


Epoch 1/5 — 119.3s  Train loss: 1.6437 Acc: 0.3927  |  Test loss: 1.2732 Acc: 0.5334
Epoch 2/5 — 117.5s  Train loss: 1.2707 Acc: 0.5413  |  Test loss: 1.0636 Acc: 0.6225
Epoch 3/5 — 119.1s  Train loss: 1.0916 Acc: 0.6131  |  Test loss: 0.9190 Acc: 0.6732
Epoch 4/5 — 122.7s  Train loss: 0.9840 Acc: 0.6529  |  Test loss: 0.8449 Acc: 0.7015
Epoch 5/5 — 119.8s  Train loss: 0.9153 Acc: 0.6784  |  Test loss: 0.7893 Acc: 0.7287

Done in 9.97 minutes. Best test acc: 0.7287

Classification report:
               precision    recall  f1-score   support

    airplane     0.6992    0.8250    0.7569      1000
  automobile     0.9029    0.8280    0.8638      1000
        bird     0.6083    0.6120    0.6102      1000
         cat     0.5894    0.5340    0.5603      1000
        deer     0.6294    0.7370    0.6789      1000
         dog     0.7541    0.5030    0.6035      1000
        frog     0.6912    0.8460    0.7608      1000
       horse     0.7538    0.7900    0.7715      1000
        ship     0

In [4]:
from google.colab import files
files.download("/content/cnn_model.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>